## RoBERTa on GoEmotions

In [21]:
# importing libraries
import numpy as np
import pandas as pd

import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaModel
from tensorflow import keras
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from RoBERTa_model_class import RobertaClassificationModel
from TextCleaner import TextCleaner

from sklearn.preprocessing import MultiLabelBinarizer

from datasets import load_dataset

In [2]:
dataset_name = "go_emotions"
dataset = load_dataset(dataset_name)
pd.set_option('display.max_colwidth', None)

# Convert to pandas dataframes
train_df = dataset["train"].to_pandas()
dev_df = dataset["validation"].to_pandas()
test_df = dataset["test"].to_pandas()

# Convert to pandas dataframes
train_df.columns = ["text", "emotions", "ids"]
dev_df.columns = ["text", "emotions", "ids"]
test_df.columns = ["text", "emotions", "ids"]

In [3]:
cleaner = TextCleaner(train_df, dev_df, test_df)
cleaner.clean_all() # Apply all cleaning steps

In [4]:
train_texts = train_df['text'].tolist()
dev_texts = dev_df['text'].tolist()
test_texts = test_df['text'].tolist()

In [5]:
len(train_df)

43410

In [6]:
train_df[:10]

,text,emotions,ids
0,my favourite food is anything i did not have to cook myself,[27],eebbqej
1,now if he does off himself everyone will think hes having a laugh screwing with people instead of actually dead,[27],ed00q6i
2,why the fuck is bayless isoing,[2],eezlygj
3,to make her feel threatened,[14],ed7ypvh
4,dirty southern wankers,[3],ed0bdzj
5,omg peyton is not good enough to help us in the playoffs dumbass broncos fans circa december 2015,[26],edvnz26
6,yes i heard about the f bombs that has to be why thanks for your reply until then hubby and i will anxiously wait 😝,[15],ee3b6wu
7,we need more boards and to create a bit more space for [NAME] then we will be good,"[8, 20]",ef4qmod
8,damn youtube and outrage drama is super lucrative for reddit,[0],ed8wbdn
9,it might be linked to the trust factor of your friend,[27],eczgv1o


In [7]:
train_labels = train_df['emotions'].tolist()
dev_labels = dev_df['emotions'].tolist()
test_labels = test_df['emotions'].tolist()

mlb = MultiLabelBinarizer()
train_labels_bin = mlb.fit_transform(train_labels)
dev_labels_bin = mlb.transform(dev_labels)
test_labels_bin = mlb.transform(test_labels)

In [8]:
train_labels_bin[:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])

In [9]:
train_labels_bin.shape

(43410, 28)

In [10]:
emotion_mapping = {
    '0' : 'admiration',
    '1' : 'amusement',
    '2' : 'anger',
    '3' : 'annoyance',
    '4' : 'approval',
    '5' : 'caring',
    '6' : 'confusion',
    '7' : 'curiosity',
    '8' : 'desire',
    '9': 'disappointment',
    '10': 'disapproval',
    '11': 'disgust',
    '12': 'embarrassment',
    '13': 'excitement',
    '14': 'fear',
    '15': 'gratitude',
    '16': 'grief',
    '17': 'joy',
    '18': 'love',
    '19': 'nervousness',
    '20': 'optimism',
    '21': 'pride',
    '22': 'realization',
    '23': 'relief',
    '24': 'remorse',
    '25': 'sadness',
    '26': 'surprise',
    '27': 'neutral'
}

target_names = list(emotion_mapping.values())
print(target_names)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [11]:
model_checkpoint = 'roberta-base'
roberta_tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

In [12]:
max_length = 128  # max length of input sequence

train_encodings = roberta_tokenizer(train_texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
dev_encodings = roberta_tokenizer(dev_texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
test_encodings = roberta_tokenizer(test_texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')

2023-11-14 14:22:41.138678: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 14:22:41.966506: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14317 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:c3:00.0, compute capability: 8.0


In [13]:
train_encodings.input_ids[1]

<tf.Tensor: shape=(128,), dtype=int32, numpy=
array([    0,  8310,   114,    37,   473,   160,  1003,   961,    40,
         206, 36279,   519,    10,  7923, 21927,   154,    19,    82,
        1386,     9,   888,  1462,     2,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,   

In [14]:
roberta_classifier = RobertaClassificationModel(checkpoint = model_checkpoint, max_length = 128)
roberta_model = roberta_classifier.model

2023-11-14 14:22:45.221775: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initial

In [15]:
# confirm all layers are frozen
roberta_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask_layer (InputLay  [(None, 128)]       0           []                               
 er)                                                                                              
                                                                                                  
 input_ids_layer (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['attention_mask_layer[0][0]',   
 el)                            thPoolingAndCrossAt               'input_ids_layer[0][0]']        
                                tentions(last_hidde                                           

In [16]:
roberta_model_history = roberta_model.fit(
    [train_encodings.input_ids, train_encodings.attention_mask],
    train_labels_bin,  # Using binarized labels
    validation_data=(
        [dev_encodings.input_ids,  dev_encodings.attention_mask],
        dev_labels_bin  # Using binarized labels
    ),
    batch_size=16,
    epochs=4
)

Epoch 1/4
2714/2714 [==============================] - 230s 80ms/step - loss: 0.1088 - accuracy: 0.4840 - val_loss: 0.0944 - val_accuracy: 0.5223
Epoch 2/4
2714/2714 [==============================] - 219s 81ms/step - loss: 0.0869 - accuracy: 0.5654 - val_loss: 0.0887 - val_accuracy: 0.5492
Epoch 3/4
2714/2714 [==============================] - 229s 84ms/step - loss: 0.0791 - accuracy: 0.6004 - val_loss: 0.0870 - val_accuracy: 0.5590
Epoch 4/4
2714/2714 [==============================] - 217s 80ms/step - loss: 0.0722 - accuracy: 0.6347 - val_loss: 0.0883 - val_accuracy: 0.5551


In [17]:
 # test the model on the test set and print the neccessary results
score = roberta_model.evaluate([test_encodings.input_ids, test_encodings.attention_mask],
                                                  test_labels_bin)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

170/170 [==============================] - 8s 49ms/step - loss: 0.0863 - accuracy: 0.5629
Test loss: 0.08629786223173141
Test accuracy: 0.5629261136054993


In [18]:
#run predict for the first three elements in the test data set
# predictions = roberta_model.predict([test_encodings.input_ids[:3], test_encodings.attention_mask[:3]])

In [19]:
#run and capture all predictions from our test set using model.predict
predictions_model1 = roberta_model.predict([test_encodings.input_ids, test_encodings.attention_mask])

threshold = 0.3
binary_predictions = (predictions_model1 > threshold).astype(int)

170/170 [==============================] - 11s 48ms/step


In [22]:
print(classification_report(test_labels_bin, binary_predictions, target_names=target_names))

                precision    recall  f1-score   support

    admiration       0.73      0.60      0.66       504
     amusement       0.83      0.76      0.80       264
         anger       0.69      0.24      0.36       198
     annoyance       0.54      0.12      0.19       320
      approval       0.74      0.15      0.25       351
        caring       0.44      0.40      0.42       135
     confusion       0.59      0.17      0.26       153
     curiosity       0.48      0.43      0.45       284
        desire       0.68      0.25      0.37        83
disappointment       0.90      0.06      0.11       151
   disapproval       0.62      0.19      0.29       267
       disgust       0.85      0.23      0.36       123
 embarrassment       0.53      0.27      0.36        37
    excitement       0.54      0.33      0.41       103
          fear       0.76      0.54      0.63        78
     gratitude       0.94      0.90      0.92       352
         grief       0.00      0.00      0.00  

/global/common/software/nersc/pm-2022q4/sw/tensorflow/2.9.0/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/global/common/software/nersc/pm-2022q4/sw/tensorflow/2.9.0/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
test_pred_labels = mlb.inverse_transform(binary_predictions) 

In [25]:
for i in range(40):
    text_example = test_texts[i]
    actual_emotion = test_df['emotions'].iloc[i]
    
    actual_labels = [emotion_mapping[str(label)] for label in actual_emotion]
    predicted_labels = [emotion_mapping[str(label)] for label in test_pred_labels[i]]

    print(f"Text: {text_example}")
    print(f"Actual Labels: {actual_labels}")
    print(f"Predicted Labels: {predicted_labels}")
    print("\n" + "="*50 + "\n")

Text: i am really sorry about your situation although i love the names sapphira cirilla and scarlett
Actual Labels: ['sadness']
Predicted Labels: ['love']


Text: it is wonderful because it is awful at not with
Actual Labels: ['admiration']
Predicted Labels: ['admiration']


Text: kings fan here good luck to you guys will be an interesting game to watch
Actual Labels: ['excitement']
Predicted Labels: []


Text: i did not know that thank you for teaching me something today
Actual Labels: ['gratitude']
Predicted Labels: ['gratitude']


Text: they got bored from haunting earth for thousands of years and ultimately moved on to the afterlife
Actual Labels: ['neutral']
Predicted Labels: ['neutral']


Text: thank you for asking questions and recognizing that there may be things that you do not know or understand about police tactics seriously thank you
Actual Labels: ['gratitude']
Predicted Labels: ['gratitude']


Text: you are welcome
Actual Labels: ['gratitude']
Predicted Labels: ['gratitud